In [14]:
import pandas as pd

train = pd.read_csv('../train.csv')
validation = pd.read_csv('../validation.csv')
test = pd.read_csv('../test.csv')
train.iloc[:, :8].to_csv('train_reduced.csv', index=False)
validation.iloc[:, :8].to_csv('validation_reduced.csv', index=False)
test.iloc[:, :8].to_csv('test_reduced.csv', index=False)

/tmp/ipykernel_236439/3087899993.py:3: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv('../train.csv')
/tmp/ipykernel_236439/3087899993.py:4: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  validation = pd.read_csv('../validation.csv')
/tmp/ipykernel_236439/3087899993.py:5: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  test = pd.read_csv('../test.csv')


In [7]:
import pandas as pd

train = pd.read_csv('train_reduced.csv')
validation = pd.read_csv('validation_reduced.csv')
test = pd.read_csv('test_reduced.csv')

database = pd.concat([train, validation, test])
database.to_csv('database.csv', index=False)

/tmp/ipykernel_236770/4211830955.py:3: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv('train_reduced.csv')
/tmp/ipykernel_236770/4211830955.py:4: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  validation = pd.read_csv('validation_reduced.csv')
/tmp/ipykernel_236770/4211830955.py:5: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  test = pd.read_csv('test_reduced.csv')


In [12]:
from plants_sm.utilities.utils import convert_csv_to_fasta

convert_csv_to_fasta('database.csv', 'sequence', 'accession', 'database.fasta')

/home/bisbii/Desktop/PHD/PlantsSM/src/plants_sm/utilities/utils.py:8: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file, sep=sep)


In [13]:
from plants_sm.alignments.alignment import BLAST

blast = BLAST("all_data_database")
blast.create_database('database.fasta')



Building a new DB, current time: 01/02/2024 18:01:26
New DB name:   /home/bisbii/Desktop/PHD/PlantsSM/examples/final_data/blast_train_validation_test/all_data_database
New DB title:  database.fasta
Sequence type: Protein
Deleted existing Protein BLAST database named /home/bisbii/Desktop/PHD/PlantsSM/examples/final_data/blast_train_validation_test/all_data_database
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 380811 sequences in 6.08584 seconds.


In [14]:
from plants_sm.utilities.utils import convert_csv_to_fasta
from plants_sm.alignments.alignment import BLAST

convert_csv_to_fasta("halogenase.csv", 'Sequence', 'Entry', 'halogenase.fasta', sep="\t")
blast = BLAST("all_data_database")

In [15]:
blast.run("halogenase.fasta", "halogenase_alignment.tsv", evalue=1e-5, num_hits=1)

In [34]:
halogenase_test_dataset = pd.read_csv("halogenase.csv", sep="\t")

In [16]:
import pandas as pd
halogenase_blast_results = pd.read_csv('halogenase_alignment.tsv', sep='\t', names= ["qseqid", "sseqid", "pident", "length", "mismatch",
                              "gapopen", "qstart", "qend", "sstart", "evalue", "bitscore"])

In [26]:
columns = list(halogenase_blast_results.columns)
i = columns.index('sseqid')
columns[i] = "accession"
halogenase_blast_results.columns = columns

In [18]:
database = pd.read_csv('database.csv')

/tmp/ipykernel_236770/726833314.py:1: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  database = pd.read_csv('database.csv')


In [29]:
database_reduced = database[database["accession"].isin(halogenase_blast_results["accession"])]
halogenase_blast_results = pd.merge(halogenase_blast_results,database_reduced,  on='accession', how='inner')

In [39]:
# get duplicated accessions
not_in_predictions = halogenase_test_dataset[~halogenase_test_dataset.loc[:, "Entry"].isin(halogenase_blast_results.loc[:, "qseqid"])]
not_in_predictions.columns = ["qseqid", "EC", "sequence"]
not_in_predictions.drop("EC", inplace=True, axis=1)

/tmp/ipykernel_236770/1130070972.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  not_in_predictions.drop("EC", inplace=True, axis=1)


In [40]:
not_in_predictions

,qseqid,sequence
0,A7KH27,MTTSGHTSFSDYSPRRRSMLLGGLGGAAALSAAGFTGMASASPRGS...
2,A0A559V0A1,MTTSGSSSVPGFSPRRRSLLLGGGSAAALTALGHAGTAAAEPGPAA...
3,A0A559V0T8,MTSGNSSSAGFSPARRSLLLGGASTAALATLGTGTAAAAGQGTGPA...
21,A0A1L1YPD7,MSNNAVSTKSALNFLDINVTEVNNYPTAIQDIIIDRRFDGMIIRGV...


In [43]:
halogenase_blast_results = pd.concat((halogenase_blast_results,not_in_predictions))
halogenase_blast_results.to_csv("halogenase_blast_results.csv", index=False)

# Price et al

In [46]:
from plants_sm.utilities.utils import convert_csv_to_fasta

convert_csv_to_fasta('price.csv', 'Sequence', 'Entry', 'price.fasta', sep="\t")
blast = BLAST("all_data_database")

In [47]:
blast.run("price.fasta", "price_alignment.tsv", evalue=1e-5, num_hits=1)

In [90]:
import pandas as pd
price_blast_results = pd.read_csv('price_alignment.tsv', sep='\t', names= ["qseqid", "sseqid", "pident", "length", "mismatch",
                              "gapopen", "qstart", "qend", "sstart", "evalue", "bitscore"])

In [91]:
columns = list(price_blast_results.columns)
i = columns.index('sseqid')
columns[i] = "accession"
price_blast_results.columns = columns

In [92]:
price_dataset = pd.read_csv("price.csv", sep="\t")

In [93]:
not_in_predictions

,qseqid,sequence
35,WP_014880051,MSQLQAPGAVVMIRPHHFCSNPETRDDNAFQTLADDTADVTSAQAQ...
63,WP_054597481,MQTTNTVLMIRPTRFSFNQDTAANNRFQRPAAAAEDVQLKALQEFD...
113,NP_436855,MPQITRRMTLSINAIVWGENIHEQTNAVVREIYPDGMHNTIAAALN...
117,NP_437726,MLISQVRKEDGSVIVAVRAPGETARAVRGAESVYALAMEAANSRRS...
140,WP_010209327,MQTTNTVLMIRPARFAFNPDTAINNRFQRQPLDPLGAQQKALEEFD...


In [94]:
database = pd.read_csv('database.csv')
database_reduced = database[database["accession"].isin(price_blast_results["accession"])]
price_blast_results = pd.merge(price_blast_results,database_reduced,  on='accession', how='inner')
# get duplicated accessions
not_in_predictions = price_dataset[~price_dataset.loc[:, "Entry"].isin(price_blast_results.loc[:, "qseqid"])]
not_in_predictions.columns = ["qseqid", "EC", "sequence"]
not_in_predictions.drop("EC", inplace=True, axis=1)
price_blast_results = pd.concat((not_in_predictions, price_blast_results), ignore_index=True)
price_blast_results.to_csv("price_blast_results.csv", index=False)

/tmp/ipykernel_236770/1146571287.py:1: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  database = pd.read_csv('database.csv')
/tmp/ipykernel_236770/1146571287.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  not_in_predictions.drop("EC", inplace=True, axis=1)
